#Fine-Tuning GPT-3

Copyright 2024 Denis Rothman

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)

Check the cost of fine-tuning your dataset on OpenAI before running the notebook.

Run this notebook cell by cell to:

1.prepare data
2.fine-tune a model
3.run a fine-tuned model
4.manage the fine-tunes

## Installing OpenAI & Wandb

Restart the runtime after installing openai and run the cell again to make sur that "import openai" is executed.

In [1]:
try:
  import openai
except:
  !pip install openai
  import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


## Your API Key

In [2]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)

#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [3]:
#remove when repository goes public
with open('drive/MyDrive/files/github.txt', 'r') as f:
    github_token = f.read().strip()

In [4]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
try:
  import wandb
except:
  !pip install wandb
  import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [6]:
!openai wandb sync

usage: openai [-h] [-v] [-b API_BASE] [-k API_KEY] [-p PROXY [PROXY ...]] [-o ORGANIZATION]
              [-t {openai,azure}] [--api-version API_VERSION] [--azure-endpoint AZURE_ENDPOINT]
              [--azure-ad-token AZURE_AD_TOKEN] [-V]
              {api,tools,migrate,grit} ...
openai: error: argument {api,tools,migrate,grit}: invalid choice: 'wandb' (choose from 'api', 'tools', 'migrate', 'grit')


# 1.Preparing the dataset

## 1.1. Preparing the data in JSON

In [7]:
#From Gutenberg to JSON
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import requests
from bs4 import BeautifulSoup
import json
import re

# First, fetch the text of the book
# Option 1: from Project Gutenberg
#url = 'http://www.gutenberg.org/cache/epub/4280/pg4280.html'
#response = requests.get(url)
#soup = BeautifulSoup(response.content, 'html.parser')

# Option 2: from the GitHub repository:
#Development access to delete when going into production
!curl -H 'Authorization: token {github_token}' -L https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-and-Computer-Vision-3rd-Edition/master/Chapter08/gutenberg.org_cache_epub_4280_pg4280.html --output "gutenberg.org_cache_epub_4280_pg4280.html"

# Open and read the downloaded HTML file
with open("gutenberg.org_cache_epub_4280_pg4280.html", 'r', encoding='utf-8') as file:
    file_contents = file.read()

# Parse the file contents using BeautifulSoup
soup = BeautifulSoup(file_contents, 'html.parser')

# Get the text of the book and clean it up a bit
text = soup.get_text()
text = re.sub('\s+', ' ', text).strip()

# Split the text into sentences
sentences = sent_tokenize(text)

# Define the separator and ending
prompt_separator = " ->"
completion_ending = "\n"

# Now create the prompts and completions
data = []
for i in range(len(sentences) - 1):
    data.append({
        "prompt": sentences[i] + prompt_separator,
        "completion": " " + sentences[i + 1] + completion_ending
    })

# Write the prompts and completions to a file
with open('kant_prompts_and_completions.json', 'w') as f:
    for line in data:
        f.write(json.dumps(line) + '\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1295k  100 1295k    0     0  2130k      0 --:--:-- --:--:-- --:--:-- 2130k


In [8]:
import pandas as pd

# Load the data
df = pd.read_json('kant_prompts_and_completions.json', lines=True)
df

,prompt,completion
0,The Project Gutenberg Etext of The Critique of...,Be sure to check the copyright laws for your ...
1,Be sure to check the copyright laws for your c...,"We encourage you to keep this file, exactly a..."
2,"We encourage you to keep this file, exactly as...",Please do not remove this.\n
3,Please do not remove this. ->,This header should be the first thing seen wh...
4,This header should be the first thing seen whe...,Do not change or edit it without written perm...
...,...,...
6122,"78-79. is their motto, under which they may le...",As regards those who wish to pursue a scienti...
6123,As regards those who wish to pursue a scientif...,"When I mention, in relation to the former, th..."
6124,"When I mention, in relation to the former, the...",The critical path alone is still open.\n
6125,The critical path alone is still open. ->,If my reader has been kind and patient enough...


##  1.2. Converting the data to JSONL

Answer the questions as necessary for your project.

In [9]:
!openai tools fine_tunes.prepare_data -f "kant_prompts_and_completions.json"

Analyzing...

- Your JSON file appears to be in a JSONL format. Your file will be converted to JSONL format
- Your file contains 6127 prompt-completion pairs
- All prompts end with suffix ` ->`
- All completions end with suffix `\n`

Based on the analysis we will perform the following actions:
- [Necessary] Your format `JSON` will be converted to `JSONL`


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `kant_prompts_and_completions_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "kant_prompts_and_completions_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` ->` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 1.44 hours to train a `cu

In [10]:
import json

# Open the file and read the lines
with open('kant_prompts_and_completions_prepared.jsonl', 'r') as f:
    lines = f.readlines()

# Parse and print the first 5 lines
for line in lines[199:300]:
    data = json.loads(line)
    print(json.dumps(data, indent=4))

{
    "prompt": "For he found that it was not sufficient to meditate on the figure, as it lay before his eyes, or the conception of it, as it existed in his mind, and thus endeavour to get at the knowledge of its properties, but that it was necessary to produce these properties, as it were, by a positive a priori construction; and that, in order to arrive with certainty at a priori cognition, he must not attribute to the object any other properties than those which necessarily followed from that which he had himself, in accordance with his conception, placed in the object. ->",
    "completion": " A much longer period elapsed before physics entered on the highway of science.\n"
}
{
    "prompt": "A much longer period elapsed before physics entered on the highway of science. ->",
    "completion": " For it is only about a century and a half since the wise Bacon gave a new direction to physical studies, or rather\u2014as others were already on the right track\u2014imparted fresh vigour t

creating the file on openai

In [11]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("/content/kant_prompts_and_completions_prepared.jsonl", "rb"),
  purpose='fine-tune'
)



FileObject(id='file-6R7NzCTTupQo6W5mW1C7NKKE', bytes=2761402, created_at=1704344300, filename='kant_prompts_and_completions_prepared.jsonl', object='file', purpose='fine-tune', status='uploaded', status_details=None)

# 2.Fine-tuning a model

In [13]:
import os
import openai
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")
from openai import OpenAI
client = OpenAI()

from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
  training_file='file-6R7NzCTTupQo6W5mW1C7NKKE',
  model="babbage-002"
)

FineTuningJob(id='ftjob-fxdOjTwoR5nuwOjgA9VDl9O4', created_at=1704344397, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-h2Kjmcir4wyGtqq1mJALLGIb', result_files=[], status='validating_files', trained_tokens=None, training_file='file-6R7NzCTTupQo6W5mW1C7NKKE', validation_file=None)

# 3.Running the fine-tuned GPT-3 model

We will now run the model for a completion task

Note: If your fine-tuned model does not appear immediately after the end of the fine-tuning process, you might have to wait until it is processed by OpenAI.

1.Go to the OpenAI Playground to test your model: https://platform.openai.com/playground

2.Then implement it in your environment

In [14]:
# List all created fine-tunes
from openai import OpenAI
client = OpenAI()
client.fine_tuning.jobs.list


<bound method Jobs.list of <openai.resources.fine_tuning.jobs.Jobs object at 0x7af852b1d600>>

[Consult OpenAI fine-tune documentation for more](https://platform.openai.com/docs/guides/fine-tuning/create-a-fine-tuned-model)